In [ ]:
import xarray as xr
import rioxarray

In [ ]:
INPUT_FILE = "path/to/your/input/file.nc" # NetCDF file used in this example
OUTPUT_FILE = "OutputCOG.tif"
LATITUDE_STR = "lat"
LONGITUDE_STR = "lon"
CRS = "4326"

In [ ]:
dset = xr.open_dataset(INPUT_FILE, decode_times=True, use_cftime=True)

# Uses rioxarray to write the coordinate reference system to the xarray dataset
dset.rio.write_crs(CRS, inplace=True)

# Rename the lat and lon from the original dataset to y and x
dset = dset.rename({LATITUDE_STR: 'y', LONGITUDE_STR: 'x'})

# This is a quick way to change the longitude from 0 - 360 degrees to -180 - 180 degrees. This is needed to correctly display the data
dset = dset.assign_coords(
    x=(((dset['x'] + 180) % 360) - 180)
)

# Sort the dataset by longitude
dset = dset.sortby('x')

In [ ]:
# Process any data variables
# For example, the dataset used for this example looked at the fraction of grid cell burnt by wildfire
calc_data = dset['burntFractionAll'].groupby('time.month').mean('time') 

In [ ]:
# Save as a Cloud Optimized GeoTIFF Geotiff data. 
calc_data.rio.to_raster(OUTPUT_FILE, driver="COG", compress="LZW")